In [15]:
import pandas as pd
import json
import ast
import plotly.express as px
import math

In [16]:
jamstones = pd.read_csv('jamstones.csv')
lavval = pd.read_csv('lavval.csv')
newagefsg = pd.read_csv('newagefsg.csv')

C:\Users\Razer\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (56) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [17]:
# Preprocessing

#helper functions
def trim_all_columns(df):
    """
    Trim whitespace from ends of each value across all series in dataframe
    """
    trim_strings = lambda x: x.strip() if isinstance(x, str) else x
    return df.applymap(trim_strings)
    
def check_format(row):
    if type(row)==str:
        if ((row[0] == '['and row[len(row)-1] == ']' and row[1] == '{'and row[len(row)-2] == '}') or (row[0] == '{'and row[len(row)-1] == '}') )and len(row)>10:
            return True
    else:
        return False

def preprocessing(df):
    df_copy = df.copy()[['id','line_items']]
    df_copy.dropna(inplace = True)
    
    df_copy = trim_all_columns(df_copy)
    df_copy.line_items = df_copy.line_items.apply(eval).apply(json.dumps)

    def clean_json(x):
        "Create apply function for decoding JSON"
        return json.loads(x)

    df_copy['line_items'] = df_copy['line_items'].apply(clean_json)  
    
    return df_copy
   


In [18]:
jamstones_preprocessed = preprocessing(jamstones)
lavval_preprocessed = preprocessing(lavval)
newagefsg_preprocessed = preprocessing(newagefsg)
items_df = pd.concat([jamstones_preprocessed, lavval_preprocessed, newagefsg_preprocessed], ignore_index=True)
# items_df = preprocessing(jamstones)

In [19]:
type(items_df['line_items'][200][0]['quantity'])

int

In [20]:
items_df

,id,line_items
0,4150394880096,"[{'id': 10681749405792, 'admin_graphql_api_id'..."
1,4150123987040,"[{'id': 10681240485984, 'admin_graphql_api_id'..."
2,4150035808352,"[{'id': 10681042796640, 'admin_graphql_api_id'..."
3,4149995405408,"[{'id': 10680952389728, 'admin_graphql_api_id'..."
4,4149958115424,"[{'id': 10680870699104, 'admin_graphql_api_id'..."
...,...,...
15738,2674319327393,"[{'id': 5785053888673, 'admin_graphql_api_id':..."
15739,2673968054433,"[{'id': 5784513413281, 'admin_graphql_api_id':..."
15740,2673190043809,"[{'id': 5783188865185, 'admin_graphql_api_id':..."
15741,2655117574305,"[{'id': 5752646762657, 'admin_graphql_api_id':..."


In [21]:
#name, quantity
#transaction set per row trans_list = [name, name2, name2 ...]
def remove_suffix(input_string, suffix):
    if suffix and input_string.endswith(suffix):
        return input_string[:-len(suffix)]
    return input_string

categories = ['bracelet', 'necklace','pendant', 'earrings', 'ring', 'lamp', 'tumble', 'display', 'pouches', 'kits', 'charm', 'bag']

def get_category(name):
    for category in categories:
        if category in name.lower():
            return category
    else:
        return 'others'
    
def items_per_transaction(row):
    num_items = len(row)
    trans_list = []
    categories_list = []
    for i in range(num_items):
        temp_quantity = row[i]['quantity']
        for j in range(temp_quantity):
            substring = '(Not found on Shopify)'
            name = row[i]['name']
            if substring in name:
                name = remove_suffix(name, substring)
            trans_list.append(name)
            category = get_category(name)
            categories_list.append(category)
    return trans_list, categories_list

def get_unique_items(some_list):
    flat_list = [item for sublist in some_list for item in sublist]
    unique_list = []
    for item in flat_list:
        if item not in unique_list:
            unique_list.append(item)
    return unique_list
    
def transaction_items(df):
    df_copy = preprocessing(df)
    line_items_list = df_copy.line_items.tolist()
    items_per_transaction_list = [items_per_transaction(row) for row in line_items_list]
    df_copy['items_per_transaction'] = items_per_transaction_list   
    
    unique_items = get_unique_items(items_per_transaction_list)
            
    return unique_items

unique_items = transaction_items(jamstones)

In [22]:
len(unique_items)

5830

In [23]:
#test

categories = ['bracelet', 'necklace','pendant', 'earrings', 'ring', 'lamp', 'tumble', 'display', 'pouches', 'kits', 'charm', 'bag']
def get_category(name):
    for category in categories:
        if category in name.lower():
            return category
    else:
        return 'others'
test = unique_items[30]
print(test)
get_category(test)

['bag', 'others', 'others', 'others']


AttributeError: 'list' object has no attribute 'lower'

In [ ]:
unique_items

['[READY PRODUCT] Crystal Bag Charm Jamstones ',
 '*PAYMENT * Live Sale $1 Listing [Jamstones Lavval NewageFSG] ',
 '[Jamstones x Lavval] Sugar & Spice Necklace (Love, Wealth and Weight-Loss) ',
 '[Jamstones x Lavval] Dubai Crystal Bracelet [Wisdom] ',
 'Career (Calmness, Healing, Wisdom, Wealth)',
 'Blue Tiger Eye Tri Pearl Necklace',
 'Phantom Garden Globe Quartz Necklace',
 'Mel (Calmness, Wisdom, Wealth, Protection)',
 'Peachy (Happiness, Self confidence, Female Wellness and Healing)',
 '[Jamstones x Lavval] Healing Crystal Tree ',
 '[Jamstones x Lavval] Positivity  - Shopee Exclusive Crystal Bracelet [Love and Fertility] ',
 '[Jamstones x Lavval] Solitaire Crystal Necklace [Choose from 20+ centerpieces] ',
 '[Jamstones x Lavval ] Selenite Tray ',
 '[Jamstones x Lavval] Love  - Shopee Exclusive Crystal Bracelet [Love & Communication] ',
 '[Jamstones x Lavval] Green Angelite  - Shopee Exclusive Crystal Bracelet [Weight Loss] ',
 '[Jamstones x Lavval] Strawberry Quartz Chips Bottle '

In [25]:
items_df = items_df[['id', 'line_items']]
line_items_list = items_df.line_items.tolist()
items_per_transaction_list = [items_per_transaction(row)[0] for row in line_items_list]
categories_per_transaction_list = [items_per_transaction(row)[1] for row in line_items_list]
items_df['items_per_transaction'] = items_per_transaction_list
items_df['categories_per_transaction'] = categories_per_transaction_list
items_df.to_csv("Items.csv")


In [ ]:
categories_per_transaction_list

[['charm', 'charm'],
 ['charm', 'charm'],
 ['others', 'others', 'others', 'others'],
 ['charm', 'charm'],
 ['necklace', 'necklace', 'bracelet'],
 ['others', 'necklace', 'necklace'],
 ['others', 'others'],
 ['others'],
 ['bracelet'],
 ['others',
  'others',
  'others',
  'others',
  'others',
  'others',
  'others',
  'others',
  'others',
  'others',
  'others',
  'others',
  'others',
  'others',
  'others',
  'others',
  'others',
  'others',
  'others',
  'others',
  'others',
  'others',
  'others',
  'others',
  'others',
  'others',
  'others',
  'others',
  'others',
  'others',
  'others',
  'others',
  'others',
  'others',
  'others',
  'others',
  'others',
  'others',
  'others',
  'others',
  'others',
  'others',
  'others',
  'others',
  'others',
  'others',
  'others',
  'others',
  'others',
  'others',
  'others',
  'others',
  'others',
  'others',
  'others',
  'others',
  'others',
  'others',
  'others',
  'others',
  'others',
  'others',
  'others',
  'others',

In [ ]:
items_df['items_per_transaction'][0]

['[READY PRODUCT] Crystal Bag Charm Jamstones ',
 '[READY PRODUCT] Crystal Bag Charm Jamstones ']

In [ ]:
# !pip install mlxtend

### MBA Using Items

In [ ]:
import mlxtend
from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()
te_ary = te.fit(items_per_transaction_list).transform(items_per_transaction_list)
te_ary   

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [ ]:
mba_df = pd.DataFrame(te_ary, columns=te.columns_)
mba_df.describe()

,$4 courier for JS5331,$50 gift voucher,*14mmLavender rose quartz solitaire,*DIRECT IMPORT from South America Palo Santo Wood Bundle,*PAYMENT * Live Sale $1 Listing,*PAYMENT * Live Sale $1 Listing [Jamstones Lavval NewageFSG,*PAYMENT * Live Sale $1 Listing [Jamstones Lavval NewageFSG],*PREMIUM* Asterism Lavender Rose Quartz 40mm Sphere,*PREMIUM* Asterism Lavender Rose Quartz 72mm Sphere,*PREMIUM* Black Tourmalinated Quartz Tumble,...,✨PROMO✨ Anti - Radiation Kit - Yes! I wish to add in the Labrdorite Crystal Grip,万事如意 (Success in everything),和气生财 (Harmony brings wealth),心想事成,心想事成 (Wish come true),心想事成 (Wish come true) - 14mm,心想事成 (Wish come true) - 16mm,恭喜发财 [Happiness and Prosperity],財源滾滾 [Endless Wealth],财源广进 (Endless fortune)
count,13939,13939,13939,13939,13939,13939,13939,13939,13939,13939,...,13939,13939,13939,13939,13939,13939,13939,13939,13939,13939
unique,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
top,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
freq,13938,13938,13938,13937,13907,13938,12771,13938,13938,13936,...,13938,13915,13934,13938,13938,13932,13936,13933,13897,13934


In [ ]:
from mlxtend.frequent_patterns import apriori

#Now, let us return the items and itemsets with at least 5% support:
apriori(mba_df, min_support = 0.01, use_colnames = True)

,support,itemsets
0,0.083794,(*PAYMENT * Live Sale $1 Listing [Jamstones La...
1,0.012196,(Wealth Magnet Bottle (U.P.$3.90) )
2,0.012124,([Jamstones x Lavval] Amethyst Eye Pillow )
3,0.011192,([Jamstones x Lavval] Healing Crystal Tree )


In [ ]:
frequent_itemsets = apriori(mba_df, min_support = 0.01, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets

,support,itemsets,length
0,0.083794,(*PAYMENT * Live Sale $1 Listing [Jamstones La...,1
1,0.012196,(Wealth Magnet Bottle (U.P.$3.90) ),1
2,0.012124,([Jamstones x Lavval] Amethyst Eye Pillow ),1
3,0.011192,([Jamstones x Lavval] Healing Crystal Tree ),1


In [ ]:
# getting th item sets with length = 2 and support more han 10%

frequent_itemsets[ (frequent_itemsets['length'] == 2) &
                   (frequent_itemsets['support'] >= 0.01) ]

,support,itemsets,length


### MBA using categories

In [ ]:
import mlxtend
from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()
te_ary2 = te.fit(categories_per_transaction_list).transform(categories_per_transaction_list)
te_ary2   

array([[False, False,  True, ..., False, False, False],
       [False, False,  True, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False,  True],
       [False, False, False, ..., False, False, False],
       [False, False, False, ...,  True, False, False]])

In [ ]:
mba_df2 = pd.DataFrame(te_ary2, columns=te.columns_)
mba_df2.describe()

,bag,bracelet,charm,display,earrings,lamp,necklace,others,pendant,ring,tumble
count,13939,13939,13939,13939,13939,13939,13939,13939,13939,13939,13939
unique,2,2,2,2,2,2,2,2,2,2,2
top,False,False,False,False,False,False,False,True,False,False,False
freq,13828,11087,13274,13931,13406,13578,12654,10082,13560,13458,13452


In [ ]:
from mlxtend.frequent_patterns import apriori

#Now, let us return the items and itemsets with at least 5% support:
apriori(mba_df2, min_support = 0.01, use_colnames = True)

,support,itemsets
0,0.204606,(bracelet)
1,0.047708,(charm)
2,0.038238,(earrings)
3,0.025899,(lamp)
4,0.092187,(necklace)
5,0.723294,(others)
6,0.027190,(pendant)
7,0.034507,(ring)
8,0.034938,(tumble)
9,0.010905,"(bracelet, necklace)"


In [ ]:
frequent_itemsets = apriori(mba_df2, min_support = 0.01, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets

,support,itemsets,length
0,0.204606,(bracelet),1
1,0.047708,(charm),1
2,0.038238,(earrings),1
3,0.025899,(lamp),1
4,0.092187,(necklace),1
5,0.723294,(others),1
6,0.027190,(pendant),1
7,0.034507,(ring),1
8,0.034938,(tumble),1
9,0.010905,"(bracelet, necklace)",2


In [ ]:
# getting th item sets with length = 2 and support more han 10%

frequent_itemsets[ (frequent_itemsets['length'] == 2) &
                   (frequent_itemsets['support'] >= 0.01) ]

,support,itemsets,length
9,0.010905,"(bracelet, necklace)",2
10,0.065930,"(others, bracelet)",2
11,0.020016,"(others, charm)",2
12,0.016644,"(others, earrings)",2
13,0.014061,"(others, lamp)",2
14,0.038382,"(others, necklace)",2
15,0.013559,"(others, ring)",2
16,0.022527,"(others, tumble)",2
